In [1]:
import wikisearch
import wikipedia
import string

In [ ]:
def search_subject(start,subs):
    start_page = wikipedia.page(start)
    closest = ("none",sys.maxsize)
    print("0 : " + wikipedia.page(start).title)
    # if 'path' in locals():
    #     if target_page not in path:
    path = [target_page]
    visited = set()
    visited.add(start)
    title = start
    exit = False
    #i = 0

    for i in range(100):
    #while not exit:
        try:

            #get next page
            current = wikipedia.page(title)
            #test for target page
            #if current not in path:
            path.append(current)
            #create model
            model = wikisearch.make_model_new(path)
            #get common words in current page
            top_20 = wikisearch.get_50_most_common(current.content)[:20]

            #test for similarity to target page
            for word,freq in top_20:
                match = []
                try:
                    for sub in subs:
                        dist = model.wv.distance(word,sub)
                        if dist < 0.2:
                            match.append(True)
                        else:
                            match.append(False)
                except KeyError:
                    pass
            if len(match) > 0 and False not in match:
                break
            search_success = True
        except wikipedia.exceptions.DisambiguationError:
            search_success = False
        except wikipedia.exceptions.PageError:
            search_success = False

        #get next link
        title = wikisearch.check_links(model,current,target,visited)
        visited.add(title)
        if i % 10 == 0:
            print(".",end="")
        #i += 1

    print("Page Found!") if exit else print("Not Found, closest page was: " + closest[0])
    print(str(i) + ": Matching page: " + title)

In [2]:
ww = wikipedia.page('Wonder Woman')
model = wikisearch.make_model_new([ww])

In [3]:
len(model.wv.vocab)

1389

In [28]:
text = ww.content.split(' ')

In [45]:
pre

4434

12

In [5]:
model.wv.distance('cheetah','villain')

0.0006876587867736816